# Export 2 videos for youtube

In [1]:
import sys
import os
import importlib
import pymongo
from dotenv import load_dotenv
# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Get the path to the parent directory
parent_dir = os.path.dirname(notebook_dir)

# Add the parent directory to sys.path if it's not already there
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [2]:
load_dotenv(override=True) 
import const
FF_FULL_PATH = os.environ['FF_FULL_PATH']

In [3]:
db_client = pymongo.MongoClient(os.environ['DB_URI'])
db = db_client['db_certificates']

In [4]:
import exam_settings
importlib.reload(exam_settings) #always get latest code

<module 'exam_settings' from '/Users/sang/Documents/Source/Python/python_webscrap/cert_exam/exam_settings/__init__.py'>

In [5]:
def main(num_of_questions, cert_symbol, channel):
    cert_metadata = exam_settings.get_cert_metadata(db, cert_symbol)
    questions = exam_settings.query_random_questions(db, num_of_questions, channel, cert_metadata)
    #print(questions)
    today_yyyymmdd = const.get_current_date_yyyymmdd()
    #export to images
    exported_result = exam_settings.generate_images(today_yyyymmdd, cert_metadata, questions)
    if exported_result == False:
        print('Cannot export all images')
        return
    #create video for questions
    exported_result = exam_settings.create_video_from_images(FF_FULL_PATH, cert_metadata['img_folder_path']+"/"+today_yyyymmdd, today_yyyymmdd+".mp4", "1/30", "/%02d.png")
    if exported_result == False:
        print('Cannot export video for questions')
        return
    #create video for answers
    exported_result = exam_settings.create_video_from_images(FF_FULL_PATH, cert_metadata['img_folder_path']+"/"+today_yyyymmdd, today_yyyymmdd+"_explan.mp4", "1/5", "/_%02d.png")
    if exported_result == False:
        print('Cannot export video for explanations')
        return
    #update db
    exam_settings.update_questions_posted(db, cert_metadata, channel, today_yyyymmdd, questions)



In [1]:
#AWS_SAA, AWS_SAP, AWS_CLF_C02, AWS_DVA_C02, AWS_MLA
#main(20, 'AWS_MLA', 'Youtube')  #export 1 video with 20 images (30 secs per each), total ~ 5min 20 secs